In [ ]:
%store -r WEAVIATE_IP
from boto3 import Session

session = Session()
credentials = session.get_credentials()
current_credentials = credentials.get_frozen_credentials()

AWS_ACCESS_KEY = current_credentials.access_key
AWS_SECRET_KEY = current_credentials.secret_key
AWS_SECRET_TOKEN = current_credentials.token
print(f"AWS_ACCESS_KEY:\t{AWS_ACCESS_KEY}")
print(f"AWS_SECRET_KEY:\t{AWS_SECRET_KEY}")
print(f"AWS_SECRET_TOKEN:\t{AWS_SECRET_TOKEN}")
print(f"WEAVIATE_IP:\t{WEAVIATE_IP}")

## Connect

In [ ]:
import weaviate

client = weaviate.connect_to_custom(
    http_host=WEAVIATE_IP, http_port="8080",  http_secure=False,
    grpc_host=WEAVIATE_IP, grpc_port="50051", grpc_secure=False,

    headers={
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
        "X-AWS-Session-Token": AWS_SECRET_TOKEN,
    },

    additional_config=weaviate.config.AdditionalConfig(timeout=(500, 1500))
)

client.is_ready()

## RAG - single prompt

In [ ]:
papers = client.collections.get("Papers")

response = papers.query.near_text("challenges of zero shot training", limit=5)

for item in response.objects:
    print(item.properties["chunk"])

In [ ]:
papers = client.collections.get("Papers")

response = papers.generate.near_text(
    "challenges of zero shot training",
    limit=5,
    single_prompt="What are the challenges of zero shot training? Please answer based on the following chunk: {chunk}"
)

for item in response.objects:
    print(item.properties["chunk"])
    print(item.generated)
    print("--------------------\n")

In [ ]:
papers = client.collections.get("Papers")

response = papers.generate.near_text(
    "zero shot training",
    limit=5,
    single_prompt="Please explain what the following chunk of text talks about in the context of zero-shot training: {chunk}"
)

for item in response.objects:
    print(item.properties["chunk"])
    print(item.generated)
    print("----------------------------------------\n")

## RAG - group task

In [ ]:
papers = client.collections.get("Papers")

response = papers.generate.near_text(
    "difficulties and challenges of zero shot training",
    limit=5,
    grouped_task="What can you tell me about zero shot training? Please answer based on the provided content",
)

for item in response.objects:
    print(item.properties["chunk"])
    
print("----------------------------------------\n")
print(response.generated)

In [ ]:
papers = client.collections.get("Papers")

response = papers.generate.near_text(
    "challenges of zero shot training",
    limit=5,
    grouped_task="What can you tell me about zero shot training? Please answer based on the provided content",
    grouped_properties=["chunk", "title"]
)

for item in response.objects:
    print(item.properties["chunk"])
    
print("----------------------------------------")
print(response.generated)

## Close the client when done

In [ ]:
client.close()